In [1]:
# 필요한 라이브러리 설치하세요

!pip install pandas
!pip install lxml
!pip install selenium
!pip install bs4

In [2]:
import pandas as pd
from selenium import webdriver
import time
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By 

driver = webdriver.Chrome(executable_path=r'./chromedriver_win32/chromedriver.exe')

df = pd.read_csv('C:\\Users\\82104\\TextMining\\12000.csv')

# 검색
for index, row in df.iterrows():
    keyword = row['검색항목']
    # 진행상황 확인용 출력
    print(index, "Place Name :", keyword)

    result = {
        '업종': None,
        '별점': None,
        '대표메뉴': None,
        '리뷰': [None] * 10
    }
    try:
        search_url = f'https://map.naver.com/v5/search/' + keyword
        driver.get(search_url)
        driver.implicitly_wait(3)
        driver.switch_to.frame('searchIframe')
        driver.implicitly_wait(3)
        try:
        # 검색결과 없을 경우
        #app-root > div > div.XUrfU > div.vEAWt > div.FYvSc
            if(driver.find_element(By.CSS_SELECTOR, 'div.FYvSc').text == '조건에 맞는 업체가 없습니다.'):
                # dataframe에 해당 ROW에 NaN 처리하고 continue
                for key in result:
                    if key == '리뷰':
                        for i in range(10):
                            df.loc[index, f'{key}_{i}'] = result[key][i]
                    else:
                        df.loc[index, key] = result[key]
                continue
        except:
            pass

        # 첫번째 가게 들어가기 ... 검색결과에서 골라야되는 4가지 상황
        #_pcmap_list_scroll_container > ul > li:nth-child(1) > div.CHC5F > a
        #_pcmap_list_scroll_container > ul > li > div.CHC5F > a > div > div > span.place_bluelink.TYaxT
        #_pcmap_list_scroll_container > ul > li:nth-child(1) > div.CHC5F > a > div > div > span.place_bluelink.TYaxT
        #_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div > a:nth-child(1) > div > div > span.place_bluelink.YwYLL
        driver.find_element(By.CSS_SELECTOR, 'span.place_bluelink').click()
        driver.implicitly_wait(3)
        driver.switch_to.default_content()
        driver.switch_to.frame('entryIframe')
        driver.implicitly_wait(3)
        html = driver.page_source 
        soup = BeautifulSoup(html, 'lxml')
    except:
        # 검색결과말고 바로 세부창 갔을 때
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame('entryIframe')
            driver.implicitly_wait(3)
            html = driver.page_source 
            soup = BeautifulSoup(html, 'lxml')
        except:
            # dataframe에 해당 ROW에 NaN 처리하고 continue
            for key in result:
                if key == '리뷰':
                    for i in range(10):
                        df.loc[index, f'{key}_{i}'] = result[key][i]
                else:
                    df.loc[index, key] = result[key]
            continue
    
    # 별점과 업종
    #app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em
    #_title > span.DJJvD
    try:
        rating = soup.select('span.PXMot.LXIwF > em')
        result['별점'] = rating[0].text
    except:
        # dataframe - rating column에 NaN 처리
        pass
    try:
        type = soup.select('span.DJJvD')
        result['업종'] = type[0].text
    except:
        # dataframe - type column에 NaN 처리
        pass

    # 대표 메뉴 1. 옵션 1 / 2. 옵션 2
    #app-root > div > div > div > div:nth-child(7) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(1) > a > div.MN48z > div.erVoL > div
    #app-root > div > div > div > div:nth-child(6) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(1) > div > div > span > a.place_bluelink.ihmWt
    try:
        menus = soup.find_all("div", "erVoL")
        if menus == []:
            menus = soup.find_all("a", "ihmWt")
        menu_texts = [menu.get_text().strip() for menu in menus]
        result['대표메뉴'] = ', '.join(menu_texts)
    #app-root > div > div > div > div:nth-child(6) > div > div:nth-child(3) > div.place_section_content > ul > li:nth-child(1) > div > div > span > a.place_bluelink.ihmWt
    except:
        # dataframe - menu column에 NaN 처리
        pass


    # 리뷰 1. 리뷰유무 확인 및 접속용 텍스트 / 2. 더보기 버튼 / 3. 리뷰 데이터
    #app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot > a.place_bluelink > em
    #app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div:nth-child(3) > div.place_section_content > ul > li:nth-child(3) > div.ZZ4OK > a > span.rvCSr > span
    #app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div:nth-child(3) > div.place_section_content > ul > li:nth-child(1) > div.ZZ4OK > a > span.zPfVt
    try:
        reviewnum = soup.select('span.PXMot > a.place_bluelink > em')
        if (reviewnum[0].text != "0"):
            driver.find_element(By.CSS_SELECTOR, 'span.PXMot > a.place_bluelink > em').click()
            driver.implicitly_wait(3)
            while True:
                try:
                    driver.find_element(By.CSS_SELECTOR, 'span.rvCSr').click()
                    time.sleep(0.1)
                except:
                    break
            html = driver.page_source 
            soup = BeautifulSoup(html, 'lxml') 
            reviews = soup.find_all("span", "zPfVt")
            review_texts = [review.get_text().strip() for review in reviews]
            # review column 0 ~ 9에 순차적으로 저장 / 없는 만큼은 NaN 처리
            for i, review in enumerate(review_texts):
                if i < 10:
                    result['리뷰'][i] = review
    except:
        # dataframe - review column에 NaN 처리
        pass
    try:
        for key, value in result.items():
            if key == '리뷰':
                for i in range(10):
                    df.loc[index, f'{key}_{i}'] = value[i]
            else:
                df.loc[index, key] = value
    except:
        continue
        

df.to_csv('주다영님12000개.csv', index=False)



C:\Users\82104\AppData\Local\Temp\ipykernel_21996\77871472.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=r'./chromedriver_win32/chromedriver.exe')


0 Place Name : 카미더가든 강남구
1 Place Name : 감성족발 강남구
2 Place Name : 밍글 종로구
3 Place Name : 미러칼국수 강남구
4 Place Name : 60계치킨서울당산역점 영등포구
5 Place Name : 강남아씨죽 광진구
6 Place Name : 우면관&수라반상 광진구
7 Place Name : 총각수산 153 강남구
8 Place Name : 용두맥주 동대문구
9 Place Name : 맛집 용산구
10 Place Name : 수남 용산구
11 Place Name : 낙천정 서초구
12 Place Name : 풍원 송파구
13 Place Name : 햄찌와베찌 뚝섬점 성동구
14 Place Name : 마구간 중구
15 Place Name : 수인등갈비 김치찜 서대문구
16 Place Name : 씨펍(C PUB) 강남구
17 Place Name : 티티카카 서대문구
18 Place Name : 카페코넌 강남구
19 Place Name : 6484 쪽갈비 서초구
20 Place Name : 에일크루 마포구
21 Place Name : 미스터스시 송파구
22 Place Name : 웰빙 엄마손 중구
23 Place Name : 한방오리집 마포구
24 Place Name : 유캔두잇 강남논현점 강남구
25 Place Name : 빅토리아레이블 송파구
26 Place Name : 하카타라멘 용산구
27 Place Name : 옹심이네 서초구
28 Place Name : 친구사이7942 영등포구
29 Place Name : 기원 중구
30 Place Name : 가마깨비 성동구
31 Place Name : 장풍 강남구
32 Place Name : 강남포장마차 성동구
33 Place Name : 전주한식 노원구
34 Place Name : 제이 강남구
35 Place Name : 타카야 강남구
36 Place Name : 얼땅쟈 동대문구
37 Place Name : 초심한우 정육식당 강남구
38 Place Name